### Google Colab utils

In [ ]:
#!pip install keras-tuner

In [ ]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [ ]:
#!kill -9 -1

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd "/gdrive/My Drive/air-pollution"

### Modeling

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Reshape, Dropout
from tensorflow.keras.layers import Bidirectional, RepeatVector, Dot, Activation
from tensorflow.keras.layers import Concatenate

from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mse
import tensorflow.keras.backend as K

from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from utils import *

import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/home/zafir/miniconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
train_encoder_input_data = np.load('./data/third-order/seq2seq/train_encoder_input_data.npy')
train_decoder_input_data = np.load('./data/third-order/seq2seq/train_decoder_input_data.npy')
train_decoder_target_data = np.load('./data/third-order/seq2seq/train_decoder_target_data.npy')

valid_encoder_input_data = np.load('./data/third-order/seq2seq/valid_encoder_input_data.npy')
valid_decoder_input_data = np.load('./data/third-order/seq2seq/valid_decoder_input_data.npy')
valid_decoder_target_data = np.load('./data/third-order/seq2seq/valid_decoder_target_data.npy')

test_encoder_input_data = np.load('./data/third-order/seq2seq/test_encoder_input_data.npy')
test_decoder_input_data = np.load('./data/third-order/seq2seq/test_decoder_input_data.npy')
test_decoder_target_data = np.load('./data/third-order/seq2seq/test_decoder_target_data.npy')

In [3]:
# due to the model architecture, we need to transform the output shape and type
train_decoder_target_data = list(np.swapaxes(train_decoder_target_data, 0, 1))
valid_decoder_target_data = list(np.swapaxes(valid_decoder_target_data, 0, 1))

In [4]:
print(train_encoder_input_data.shape)
print(train_decoder_input_data.shape)
print(len(train_decoder_target_data), train_decoder_target_data[0].shape)

(67396, 24, 23)
(67396, 12, 23)
12 (67396, 2)


In [5]:
Tx, encoder_input_dim = (train_encoder_input_data.shape[1], 
                         train_encoder_input_data.shape[2])
    
Ty, decoder_input_dim = (train_decoder_input_data.shape[1], 
                         train_decoder_input_data.shape[2])

# we are predicting the pollution only, leave out the mask
decoder_output_dim = 1  

In [6]:
batch_size = 64
epochs = 250
max_trials = 250
executions_per_trial = 1
patience = 20

In [7]:
def one_step_attention(encoder_outputs, h_prev, attention_repeat, 
                       attention_concatenate, attention_dense_1,
                       attention_dense_2, attention_activation,
                       attention_dot):
    """
    Performs one step of attention: Outputs a context vector computed 
    as a dot product of the attention weights "alphas" and the outputs
    of the encoder.
    
    Arguments:
    encoder_outputs -- outputs of the encoder, numpy-array of shape 
                       (m, Tx, 2*encoder_latent_dim)
    h_prev -- previous hidden state of the decoder LSTM, numpy-array 
              of shape (m, decoder_latent_dim)
    attention_repeat -- predefined repeat layer for the attention
    attention_concatenate -- predefined concatenate layer for the 
                             attention
    attention_dense_1 -- predefined dense layer for the attention
    attention_dense_2 -- predefined dense layer for the attention
    attention_activation -- predefined activation layer for the 
                            attention
    attention_dot -- predefined dot layer for the attention
    
    
    Returns:
    context -- context vector, input to the decoder LSTM cell
    """
    
    # Repeat h_prev to be of shape (m, Tx, decoder_latent_dim) 
    h_prev = attention_repeat(h_prev)
    
    # Concatenate the encoder outputs to the repeated vector
    concat = attention_concatenate([encoder_outputs, h_prev])
       
    # Compute the energies
    energies = attention_dense_1(concat)
    energies = attention_dense_2(energies)

    # Compute the alphas by applying softmax on the energies
    alphas = attention_activation(energies)

    # Compute the context vector by dotting the alphas and 
    # the encoder outputs
    context = attention_dot([alphas, encoder_outputs])
    
    return context

In [8]:
class AttentionModel(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, decoder_input_dim, 
                 decoder_output_dim):
        """
        Constructor for the derived HyperModel class
        
        Arguments:
        Tx -- length of the input sequence
        Ty -- length of the output sequence
        encoder_input_dim -- length of input vector for the encoder
        decoder_input_dim -- length of input vector for the decoder
        decoder_output_dim -- length of output vector for the decoder
        """
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim


    def build(self, hp):
        """
        Builds a seq2seq LSTM model with an attention mechanism. 

        Arguments:
        hp -- hyperparameters object from keras-tuner

        Returns:
        model -- Keras model instance
        """

        # ------------------- SHARED LAYERS ---------------------
        encoder_latent_dim = hp.Int('encoder_latent_dim', min_value=16, max_value=48, 
                                    step=16)
        decoder_latent_dim = 2 * encoder_latent_dim
        attention_dense_dim = hp.Int('attention_dense_dim', min_value=8, max_value=14, 
                                     step=2)

        # Encoder layers
        encoder_lstm = Bidirectional(LSTM(encoder_latent_dim, return_sequences=True, 
                                          name='encoder_lstm'), merge_mode='concat')

        # Attention layers
        attention_repeat = RepeatVector(self.Tx, name='attention_repeat')
        attention_concatenate = Concatenate(axis=-1, name='attention_concatenate')
        attention_dense_1 = Dense(attention_dense_dim, activation='tanh', 
                                  name='attention_dense_1')
        attention_dense_2 = Dense(1, activation='relu', name='attention_dense_2')
        attention_activation = Activation(softmax, name='attention_activation') 
        attention_dot = Dot(axes = 1)

        # Decoder layers
        decoder_concatenate = Concatenate(axis=-1, name='decoder_concatenate')
        decoder_lstm = LSTM(decoder_latent_dim, return_state=True, 
                            name='decoder_lstm')
        decoder_dense = Dense(self.decoder_output_dim, activation='linear',
                              name='decoder_dense')

        # ---------------------- MODEL ------------------------
        # Encoder inputs
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')
        encoder_outputs = encoder_lstm(encoder_inputs)

        # Decoder inputs
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Zeros tensors as initial values for h and c.
        # Basically, I apply the decoder LSTM on the first timestep of encoder outputs  
        # concatenated with decoder inputs in order to get the hidden states h and c, 
        # and then I create zeros tensors from their shape, because I cannot obtain 
        # the batch size dynamically. Moreover, I have to apply an identity lambda 
        # function in order to cast the zero tensor to a Keras tensor (otherwise it 
        # cannot be passed as initial_state)

        # x is a slice of the encoder outputs
        x = Lambda(lambda x: x[:, 0, :])(encoder_outputs)
        x = K.expand_dims(x, axis=1)
        # y is a slice of the decoder inputs
        y = Lambda(lambda y: y[:, 0, :])(decoder_inputs)
        y = K.expand_dims(y, axis=1)
        # z is a concatenation of x and y
        z = Concatenate(axis=-1)([x, y])
        # we feed the dummy tensor in order to obtain a sample tensor of h and c
        _, h, c = decoder_lstm(z)
        # create tensors of zeros using the shapes of the previous dummies
        h = Lambda(lambda x: x, name='h0')(K.zeros_like(h))
        c = Lambda(lambda x: x, name='c0')(K.zeros_like(c))

        # Decoder outputs
        outputs = []

        for t in range(self.Ty):
            # Perform attention for one timestep
            context = one_step_attention(encoder_outputs, h, attention_repeat, 
                                         attention_concatenate, attention_dense_1,
                                         attention_dense_2, attention_activation,
                                         attention_dot)
            
            # Concatenate the context vector and the decoder input
            decoder_input = Lambda(lambda x: x[:, t, :])(decoder_inputs)
            decoder_input = K.expand_dims(decoder_input, axis=1)
            full_decoder_input = decoder_concatenate([context, decoder_input])

            # Apply post-attention LSTM
            h, _, c = decoder_lstm(inputs=full_decoder_input, initial_state=[h, c])

            # Apply dense to compute the output
            decoder_output = decoder_dense(h)
            
            # Append the the model's outputs
            outputs.append(decoder_output)

        model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=attention_masked_mse)
        return model

In [9]:
model_builder = AttentionModel(Tx, Ty, encoder_input_dim, decoder_input_dim,
                                decoder_output_dim)
tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner/attention', 
                     project_name='test')

INFO:tensorflow:Reloading Oracle from existing project local-keras-tuner/attention/test/oracle.json


In [10]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 67396 samples, validate on 3388 samples
Epoch 1/250


 1344/67396 [..............................] - ETA: 1:52:32 - loss: 16.0021 - decoder_dense_loss: 1.1840 - decoder_dense_1_loss: 1.2867 - decoder_dense_2_loss: 1.3426 - decoder_dense_3_loss: 1.5244 - decoder_dense_4_loss: 1.4441 - decoder_dense_5_loss: 1.4496 - decoder_dense_6_loss: 1.4035 - decoder_dense_7_loss: 1.2620 - decoder_dense_8_loss: 1.1125 - decoder_dense_9_loss: 1.1917 - decoder_dense_10_loss: 1.3735 - decoder_dense_11_loss: 1.427 - ETA: 57:07 - loss: 13.9686 - decoder_dense_loss: 0.9185 - decoder_dense_1_loss: 0.9271 - decoder_dense_2_loss: 0.9632 - decoder_dense_3_loss: 1.0529 - decoder_dense_4_loss: 1.0734 - decoder_dense_5_loss: 1.2203 - decoder_dense_6_loss: 1.2647 - decoder_dense_7_loss: 1.2765 - decoder_dense_8_loss: 1.2466 - decoder_dense_9_loss: 1.2357 - decoder_dense_10_loss: 1.3824 - decoder_dense_11_loss: 1.4072  - ETA: 38:33 - loss: 13.8532 - decoder_dense_loss: 0.7992 - decoder_dense_1_loss: 0.7715 - decoder_dense_2_loss: 0.8116 - decoder_dense_3_loss: 0.9666 

 2176/67396 [..............................] - ETA: 6:41 - loss: 4.4712 - decoder_dense_loss: 0.4186 - decoder_dense_1_loss: 0.3081 - decoder_dense_2_loss: 0.3009 - decoder_dense_3_loss: 0.3295 - decoder_dense_4_loss: 0.3371 - decoder_dense_5_loss: 0.3578 - decoder_dense_6_loss: 0.3827 - decoder_dense_7_loss: 0.3881 - decoder_dense_8_loss: 0.4000 - decoder_dense_9_loss: 0.4001 - decoder_dense_10_loss: 0.4202 - decoder_dense_11_loss: 0.428 - ETA: 6:27 - loss: 4.3694 - decoder_dense_loss: 0.4155 - decoder_dense_1_loss: 0.3033 - decoder_dense_2_loss: 0.2947 - decoder_dense_3_loss: 0.3199 - decoder_dense_4_loss: 0.3277 - decoder_dense_5_loss: 0.3505 - decoder_dense_6_loss: 0.3726 - decoder_dense_7_loss: 0.3812 - decoder_dense_8_loss: 0.3912 - decoder_dense_9_loss: 0.3896 - decoder_dense_10_loss: 0.4079 - decoder_dense_11_loss: 0.415 - ETA: 6:14 - loss: 4.2531 - decoder_dense_loss: 0.4054 - decoder_dense_1_loss: 0.3023 - decoder_dense_2_loss: 0.2909 - decoder_dense_3_loss: 0.3128 - decoder_

KeyboardInterrupt: 